<center><img src=molecule.png width=800px height=30%>

 # <center> <strong> Kaggle: Predicting a Biological Response

### Вводные данные
Наша практика будет основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа). Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV. Каждая строка представляет молекулу.

* Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];
* Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.
Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

Загружаем все необходимые библиотеки для дальнейшей работы

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from sklearn import model_selection
import optuna



ModuleNotFoundError: No module named 'numpy'

Записываем датасет в переменную data

In [2]:
data=pd.read_csv('_train_sem09__1_.zip')

: 

Выведем на экран 5 строк из нашего датасета, что бы посмотреть из чего состоит он

In [ ]:
data.head(5)

В задаче указано, что данные подготовленные, но для уверенности смотрим нет ли  пропусков в данных

In [ ]:
data.isna().sum().sum()

И дубликатов

In [ ]:
data.duplicated().sum()

Записываем в переменные X и y нашу матрицу наблюдений и целевой признак

In [6]:
X=data.drop(['Activity'],axis=1)
y=data['Activity']

Разделим данные на тренировочные и тестовые

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

При этом не будем стратифицировать их, т.к. далее мы будем использовать это при оценке нашей модели. Рассмотрим распределение в целевом признаке

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

Данные почти одинаково распределены. И миноритальный класс в обоих выборках распределен почти идентично

### 1.1  Подбор лучших гиперпараметров с помощью GridSearch для модели LogisticRegression

Введем наши переменные для дальнейшей работы, KFOLD стратификацию и параметры для поиска лучших из них, отдельно для LogisticRegression и для RandomForest

In [10]:
cvs=StratifiedKFold(n_splits=5)

In [11]:
params_lr=[{'penalty':['l1','l2'],
        'solver':['liblinear','saga'],
        'C':list(np.arange(0,1.1,0.25))},
        {'penalty':['l2',None],
        'solver':['lbfgs','sag'],
        'C':list(np.arange(0,1.1,0.25))}]
params_rf={'n_estimators':list(range(100,401,100)),
        'criterion':['entropy','gini'],
        'max_depth':[3,5,7],
        'min_samples_leaf':[3,5,7]}

Запустим нашу функцию GridSearch и выясним, какое время занимает поиск лучших параметров

In [ ]:
model_logic_GSCV=LogisticRegression(random_state=42,max_iter=200)
GridSearch_lr=GridSearchCV(estimator=model_logic_GSCV,
             param_grid=params_lr,
             n_jobs=-1,
             scoring='f1',
             cv=cvs)
%time GridSearch_lr.fit(X_train,y_train)

Данная функция, путем перебора всех вариантов, которые мы ей предложили справилась за 7 минут 19 секунд

Далее выведем на экран лучшие параметры, которая функция предложила и подставим их в модель LogisticRegression

In [ ]:
print('Параметры, полученные с помощью GridSearch для логистической регрессии')
print()
print('Лучшие параметры:{}'.format(GridSearch_lr.best_params_))
print('f1-score для валидационной выборки: {}'.format(GridSearch_lr.best_score_.round(3)))

In [14]:
C=GridSearch_lr.best_params_['C']
penalty=GridSearch_lr.best_params_['penalty']
solver=GridSearch_lr.best_params_['solver']

In [ ]:
model_logic=LogisticRegression(C=C,penalty=penalty,solver=solver,random_state=42,max_iter=200)
model_logic.fit(X_train,y_train)
predict_logic_y_train=model_logic.predict(X_train)
predict_logic_y_test=model_logic.predict(X_test)
print('Результат, полученный подбором гиперпараметров для логистической регрессии, с помощью GridSearch')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_logic_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_logic_y_test).round(3)))

Будем в дальнейшем сравивать время обучения функций поиска гиперпараметров между собой,сами гиперпараметры и результат

### 1.2 Подбор лучших гиперпараметров с помощью GridSearch для модели RandomForest

Проделаем тоже самое для модели RandomForest

In [ ]:
model_Random_Forest_GSCV=RandomForestClassifier(random_state=42)

GridSearch_rf=GridSearchCV(estimator=model_Random_Forest_GSCV,
             param_grid=params_rf,
             n_jobs=-1,
             scoring='f1',
             cv=cvs
)

%time GridSearch_rf.fit(X_train,y_train)

3 минуты 21 секунда, нашей функции GridSearch понадобилось, что бы справится с поиском лучших гиперпараметров для модели RandomForest.Выводим лучший вариант и подставляем их в модель, далее смотрим метрику f1 уже с подобранными гиперпараметрами.

In [ ]:
print('Параметры, полученные с помощью GridSearch для случайного леса')
print()
print('Лучшие параметры для случайного леса:{}'.format(GridSearch_rf.best_params_))
print('f-score для валидационной выборки: {}'.format(GridSearch_rf.best_score_.round(3)))

In [18]:
criterion=GridSearch_rf.best_params_['criterion']
max_depth=GridSearch_rf.best_params_['max_depth']
min_samples_leaf=GridSearch_rf.best_params_['min_samples_leaf']
n_estimators=GridSearch_rf.best_params_['n_estimators']

In [ ]:
model_Random_Forest=RandomForestClassifier(criterion=criterion,max_depth=max_depth,min_samples_leaf=min_samples_leaf,n_estimators=n_estimators,random_state=42)

model_Random_Forest.fit(X_train,y_train)

predict_RF_y_train=model_Random_Forest.predict(X_train)
predict_RF_y_test=model_Random_Forest.predict(X_test)

print('Результат, полученный подбором гиперпараметров для случайного леса, с помощью GridSearch')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_RF_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_RF_y_test).round(3)))


Данная модель более точно предсказала целевой признак на тестовых данных(0.802 против 0.789), но эта разница не значительная. Но все же есть. И в два раза быстрее

### 2.1 Подбор лучших гиперпараметров с помощью RandomizeSearch для LogisticRegression

Так же, используя все теже самые гиперпараметры будем искать лучшие с помощью функции поиска RandomizeSearch

In [ ]:
model_logic_RSCV=LogisticRegression(random_state=42,max_iter=200)
RandomizedSearch_lr=RandomizedSearchCV(estimator=model_logic_RSCV,
                                       param_distributions=params_lr,
                                       scoring='f1',
                                       n_jobs=-1,
                                       n_iter=10)
%time RandomizedSearch_lr.fit(X_train,y_train)

Данная модель справляется с задачей за 3 минуты 2 секунды. Что намного быстрее чем модель, которую мы использовали ранее. Выводим на экран результат, далее подставляем их в нашу модель LogisticRegression

In [ ]:
print('Параметры, полученные с помощью RandomizeSearch для логистической регрессии')
print()
print('Лучшие параметры для логистической регрессии:{}'.format(RandomizedSearch_lr.best_params_))
print('f-score для валидационной выборки: {}'.format(RandomizedSearch_lr.best_score_.round(3)))

Но данная модель посчитала, что лучше будет использовать гиперпараметр solver ни 'liblinear', а  'saga'. Хотя если верить рекомендациям использования данного параметра 'liblinear' данный выбор хорош для небольших датасетов.

In [22]:
C=RandomizedSearch_lr.best_params_['C']
penalty=RandomizedSearch_lr.best_params_['penalty']
solver=RandomizedSearch_lr.best_params_['solver']

Подставим наши гиперпараметры в модель и посмотрим метрику на тестовых данных.Сравним, смогли ли мы засчет скорости не потерять качество модели.

In [ ]:
model_logic=LogisticRegression(C=C,penalty=penalty,solver=solver,random_state=42,max_iter=200)
model_logic.fit(X_train,y_train)
predict_logic_y_train=model_logic.predict(X_train)
predict_logic_y_test=model_logic.predict(X_test)
print('Результат, полученный подбором гиперпараметров для логистической регрессии, с помощью RandomizeSearch')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_logic_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_logic_y_test).round(3)))

Мы видим, что при уменьшении времени в два раза мы не потеряли качество модели, а даже ее улучшили немного. Тем самым модель чуть лучше нашала предугадывать класс.

### 2.2 Подбор лучших гиперпараметров с помощью RandomizeSearch для RandomForest

Проверим, так же это работает при использовании модели RandomForest?

In [ ]:
model_Random_Forest_RSCV=RandomForestClassifier(random_state=42)

RandomizedSearch_rf=RandomizedSearchCV(estimator=model_Random_Forest_RSCV,
             param_distributions=params_rf,
             n_jobs=-1,
             scoring='f1',
             cv=cvs
)

%time RandomizedSearch_rf.fit(X_train,y_train)

Модель справилась за секунды(36), что почти в три раза быстрее чем модель, которая перебирает все варианты гиперпараметов.

In [ ]:
print('Параметры, полученные с помощью RandomizeSearch для случайного леса')
print()
print('Лучшие параметры для случайного леса:{}'.format(RandomizedSearch_rf.best_params_))
print('f-score для валидационной выборки: {}'.format(RandomizedSearch_rf.best_score_.round(3)))

In [26]:
criterion=RandomizedSearch_rf.best_params_['criterion']
max_depth=RandomizedSearch_rf.best_params_['max_depth']
min_samples_leaf=RandomizedSearch_rf.best_params_['min_samples_leaf']
n_estimators=RandomizedSearch_rf.best_params_['n_estimators']

Подставим наши данные в модель и узнаем результат.

In [ ]:
model_Random_Forest=RandomForestClassifier(criterion=criterion,max_depth=max_depth,min_samples_leaf=min_samples_leaf,n_estimators=n_estimators,random_state=42)

model_Random_Forest.fit(X_train,y_train)

predict_RF_y_train=model_Random_Forest.predict(X_train)
predict_RF_y_test=model_Random_Forest.predict(X_test)

print('Результат, полученный подбором гиперпараметров для случайного леса, с помощью RandomizeSearch')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_RF_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_RF_y_test).round(3)))

Данная модель поиска гиперпараметров переплюнула результат, который был ранее, не особо много, но время обучения поиска гиперпараметров зато при этом мы снизили почти в три раза, что не может не радовать.

### 3.1 Подбор лучших гиперпараметров с помощью Tree-Structured Parzen Estimators (TPE) для LogisticRegression 

Итак дошли до более сложных моделей подбора гиперпараметров. Для начало нам надо задать словарь всех вариантов наших гиперпараметров для модели и написать функцию, которая будет возвращать метрику, но т.к. данная функция поиска гиперпараметров основана на минимизации, то нам надо поставить знак минус перед метрикой.

In [28]:
params_lr_TPE={'penalty':hp.choice('penalty',['l1','l2']),
            'solver':hp.choice(label='solver',options=['liblinear','saga']),
            'C':hp.uniform('C',0,1)}


def hyperopt_rf(params=params_lr_TPE, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty':str(params['penalty']), 
              'solver': str(params['solver']), 
             'C': float(params['C'])}
    # используем эту комбинацию для построения модели
    model = LogisticRegression(**params, random_state=42,max_iter=200)
    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


И применим нашу написанную функцию в качестве гиперпараметра уже функции минимизации. Так же будем записывать наши результаты в переменную trials, что бы в дальнейшем программа запоминала лучшие результаты, когда мы будем подставлять уже другие гиперпараметры

In [ ]:
trials = Trials() # используется для логирования результатов
%time
best=fmin(hyperopt_rf, # наша функция FMIN
    space=params_lr_TPE, # пространство гиперпараметров
    algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно TPE
    max_evals=20, # максимальное количество итераций
    trials=trials, # логирование результатов   TRIALS
    rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
)

Самый долгий по времени алгоритм обучения и поиска лучших гиперпараметров 14 минут 8 секунд. На первый взгляд это самый не интересный алгоритм, но суть его в дальнейшем использовании. Т.к. он запоминает наилучшие варианты и все комбинации, которые были ранее. ПРИ ЭТОМ ОБЯЗАТЕЛЬНО ИСПОЛЬЗОВАТЬ ГИПЕРПАРАМЕТР TRIALS ТОТ ЖЕ, ЧТО И РАНЕЕ ДЛЯ ЭТОЙ МОДЕЛИ, ЧТО БЫ МОДЕЛЬ НАЧАЛА ИСКАТЬ ВАРИАНТ УЖЕ С НОВЫМИ ДАННЫЕ, НО ПОМНИВ ЛУЧШИЙ РЕЗУЛЬТАТ, КОТОРЫЙ БЫЛ ДОСТИГНУТ РАНЕЕ.

In [ ]:
print('Параметры, полученные с помощью HYPEROPT для Логистической регрессии')
print()
print('Лучшие параметры для логистической регрессии:{}'.format(best))


В итоге мы получили 'C': 0.4805571687112412, 'penalty': 'l1', 'solver': 'saga'. Подставим эти гиперпараметры в нашу модель LogisticRegression

In [ ]:
model_logic=LogisticRegression(C=best['C'],penalty='l1',solver='saga',random_state=42,max_iter=200)
model_logic.fit(X_train,y_train)
predict_logic_y_train=model_logic.predict(X_train)
predict_logic_y_test=model_logic.predict(X_test)
print('Результат, полученный подбором гиперпараметров для логистической регрессии, с помощью HyperOpt')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_logic_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_logic_y_test).round(3)))

Как результат мы видим, что он уступил модели RandomizeGrid, но там мы использовали еще один вариант гиперпараметров и сейчас мы его добавим в наши поиски

In [32]:
params_lr_TPE_2={'penalty':hp.choice('penalty','l2'),
            'solver':hp.choice(label='solver',options=['lbfgs','sag']),
            'C':hp.uniform('C',0,1)}


def hyperopt_rf(params=params_lr_TPE_2, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty':str(params['penalty']), 
              'solver': str(params['solver']), 
             'C': float(params['C'])}
    # используем эту комбинацию для построения модели
    model = LogisticRegression(**params, random_state=42,max_iter=200)
    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

Добавили новые сочитания и запускаем поиск. При этом параметр Trials мы оставляем тем же.

In [ ]:
best=fmin(hyperopt_rf, # наша функция FMIN
          space=params_lr_TPE, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно TPE
          max_evals=40, # максимальное количество итераций
          trials=trials, # логирование результатов   TRIALS
          rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
)          

И после добавления новых данных модель уже справилась с подбором и сравнением лучших комбинаций гиперпараметров за 9 секунд.

In [ ]:
print('Параметры, полученные с помощью HYPEROPT для Логистической регрессии')
print()
print('Лучшие параметры для логистической регрессии:{}'.format(best))

In [ ]:
model_logic=LogisticRegression(C=best['C'],penalty='l2',solver='lbfgs',max_iter=200,random_state=42)
model_logic.fit(X_train,y_train)
predict_logic_y_train=model_logic.predict(X_train)
predict_logic_y_test=model_logic.predict(X_test)
print('Результат, полученный подбором гиперпараметров для логистической регрессии, с помощью HyperOpt')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_logic_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_logic_y_test).round(3)))

* В итоге мы получили метрику, примерно одинаковую со всеми моделями, которые были использованы ранее, но за более короткое время, если мы будем не однократно искать нужные параметры, а подбирать все разные и разные.

### 3.2 Подбор лучших гиперпараметров с помощью Tree-Structured Parzen Estimators (TPE) для RandomForest

Так же все гиперпараметры и варианты их значений выводим в отдельную переменную и используя функцию возвращаем метрику со знаком минус.

In [36]:
params_rf_TPE={'n_estimators':hp.quniform('n_estimators',100,400,100),
            'criterion':hp.choice(label='criterion',options=['entropy','gini']),
            'max_depth':hp.quniform('max_depth',3,7,2),
            'min_samples_leaf':hp.quniform('min_samples_leaf',3,7,2)}
             

def hyperopt_rf(params=params_lr_TPE, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators':int(params['n_estimators']), 
              'criterion': str(params['criterion']), 
              'max_depth': int(params['max_depth']),
              'min_samples_leaf': int(params['min_samples_leaf'])}
    # используем эту комбинацию для построения модели
    model = RandomForestClassifier(**params, random_state=42)
    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

А вот если у нас поменялась модель, то уже старый trials мы уже не можем использовать, только новый.

In [ ]:
trials_rf = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция FMIN
          space=params_rf_TPE, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно TPE
          max_evals=20, # максимальное количество итераций
          trials=trials_rf, # логирование результатов   TRIALS
          rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
)

Модель справилась за 1 минуту 55 секунд, меньше чем через GridSearch

In [ ]:
print('Параметры, полученные с помощью HYPEROPT для Логистической регрессии')
print()
print('Лучшие параметры для логистической регрессии:{}'.format(best_rf))

In [ ]:
model_Random_Forest=RandomForestClassifier(criterion='entropy',max_depth=6,min_samples_leaf=4,n_estimators=200,random_state=42)
model_Random_Forest.fit(X_train,y_train)
predict_Random_y_train=model_Random_Forest.predict(X_train)
predict_Random_y_test=model_Random_Forest.predict(X_test)
print('Результат, полученный подбором гиперпараметров для логистической регрессии, с помощью HyperOpt')
print()
print('Метрика F1-score на тренировочных данных равна:{}'.format((metrics.f1_score(y_train,predict_Random_y_train)).round(3)))
print('Метрика F1-score на тестовых данных равна:{}'.format(metrics.f1_score(y_test,predict_Random_y_test).round(3)))

В итоге получились значения примерно одинаковые, как и получали ранее.Но можно рассмотреть еще другие варианты. Что мы делали ранее. Но лучше мы посмотрим на график, который показывает, как менялась метрика на протяжении всей работы модели по оптимизации гиперпараметров.

In [ ]:
# отрисуем, как менялась точность при различных гиперпараметрах
tpe_results=np.array([[x['result']['loss'],
                      x['misc']['vals']['criterion'][0],
                      x['misc']['vals']['max_depth'][0],
                      x['misc']['vals']['n_estimators'][0],
                      x['misc']['vals']['min_samples_leaf'][0]] for x in trials_rf.trials])

tpe_results_df=pd.DataFrame(tpe_results,
                           columns=['score', 'criterion', 'max_depth','n_estimators','min_samples_leaf'])
# тепловая карта в данном случае не очень наглядна, возьмем линейный график
tpe_results_df.plot(subplots=True,figsize=(10, 10));

На графике отчетливо видно, что минимальное значение метрики находится чуть дальше отметки 2.5 по оси X. Если провести мысленно вертикальную черту, то на ней окажутся значения следующие:
criterion='entropy',
max_depth=6,
n_estimators=200,
min_samples_leaf=4
Что и выдала программа

### 4.1 Подбор лучших гиперпараметров с помощью OPTUNA для LogisticRegression

In [41]:
def optuna_lg(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l1','l2'])
  solver = trial.suggest_categorical('solver', ['liblinear','saga'])
  C= trial.suggest_float('C', low=0,high=1)
  # создаем модель
  model = LogisticRegression(penalty=penalty,
                            solver=solver,
                            C=C,
                            max_iter=200,
                            random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))
  return score

In [ ]:
import optuna
%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_lr = optuna.create_study(study_name="LinearRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr.optimize(optuna_lg, n_trials=20)

И так время обучения модели по оптимизации равно 3 минуты 8 секунд. Достаточно неплоха, по сравнению Hyperopt, посмотрим на качество.

На обучающей выборке модель показала высокий результат.

In [ ]:
# выводим результаты на обучающей выборке
print(study_lr.best_params) # Наилучшие значения гиперпараметров 
print(study_lr.best_value) #"f1_score на обучающем наборе

In [ ]:
# рассчитаем точность для тестовой выборки
model = LogisticRegression(**study_lr.best_params,random_state=42,max_iter=200 )
model.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.3f}".format(model.score(X_test,y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test,y_test_pred)))

Результат действительно у всех моделей более менее схож. Вопрос только в том, как дальше будут использоваться данные модели, либо вариации добавляться либо разово.

Сформируем график, где мы увидим, растет ли наша метрика с увеличением итераций.Для начало проверим, можем ли мы воспользоваться данным инструменом.

In [ ]:
optuna.visualization.is_available()

Все отлично, визуализируем зависимость метрики от количества итераций.

In [ ]:
optuna.visualization.plot_optimization_history(study_lr, target_name="f1_score")

<img src=Optuna_lr.png width=1450px height=30%>

Видим, что наша метрика на тренировочных данных действительно растет.

Далее рассмотрим важность гиперпараметов и посмотрим так же в каких значениях метрика максимальная.

In [ ]:
optuna.visualization.plot_param_importances(study_lr, target_name="f1_score")

<img src=param_importances_Optuna_lr.png width=1450px height=30%>

Самым важным параметром оказался C.Это штраф за переобучение модели.

In [ ]:
optuna.visualization.plot_contour(study_lr, params=["C", "penalty"],
                                  target_name="f1_score")

<img src=contour_Optuna_lr.png width=1450px height=30%>

А вот значения самих параметров С чуть меньше 0.2, ранее в моделях наблюдалось повышение данного параметра, тут видим наоборот и penalty l1. Примерно такие же данные показала программа оптимизации HyperOpt. 

### 4.2 Подбор лучших гиперпараметров с помощью OPTUNA для RandomForest

In [49]:

def optuna_rf(trial):
    n_estimators=trial.suggest_int('n_estimators',low=100,high=400)
    criterion=trial.suggest_categorical('criterion', ['entropy','gini'])
    max_depth=trial.suggest_int('max_depth',low=3,high=10)
    min_samples_leaf=trial.suggest_int('min_samples_leaf',low=3,high=10)

    model_R=RandomForestClassifier(n_estimators=n_estimators,  
                                               criterion=criterion,
                                               max_depth=max_depth,
                                               min_samples_leaf=min_samples_leaf,
                                               random_state=42)
    model_R.fit(X_train,y_train)
    predict_X=model_R.predict(X_train)
    score=metrics.f1_score(y_train,predict_X)
    return score
    

In [ ]:
import optuna
%time
study_random=optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study_random.optimize(optuna_rf,n_trials=20)

Оптимизация optuna показывает отличное время если модель RandomForest. Время обучения составило чуть меньше минуты. 

In [ ]:
# выводим результаты на обучающей выборке
print(study_random.best_params) # Наилучшие значения гиперпараметров 
print(study_random.best_value) #"f1_score на обучающем наборе

In [ ]:
# рассчитаем точность для тестовой выборки
model_rf = RandomForestClassifier(**study_random.best_params,random_state=42)
model_rf.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.3f}".format(model_rf.score(X_test,y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test,y_test_pred)))

In [ ]:
optuna.visualization.plot_optimization_history(study_random, target_name="f1_score")

<img src=optimization_history_rf_Optuna.png width=1450px height=30%>

По графику так же видим, что при увеличении итераций наша метрика растет.

In [ ]:
optuna.visualization.plot_ccccccccccc(study_random, target_name="f1_score")

<img src=param_importances_Optuna_rf.png width=1450px height=30%>

Максимальная зависимость от гиперпараметра max_depth

In [ ]:
optuna.visualization.plot_contour(study_random, params=["n_estimators", "max_depth"],
                                  target_name="f1_score")

<img src=contour_Optuna_rf.png width=1450px height=30%>

Так же на контурной карте видим, что максимальные значения достигаются при n_estimators=250 и max_depth=10

Вывод: Все модели имеют право на жизнь и на активное использование, т.к. достигают одиннаковые метрики. Вопрос только во временных рамках, но для удобства и с визуализацией отлично подойдет optuna. Так же она берет не те числа, которые мы сами вписываем или тот шаг, а рандомные, что может улучшить качество модели. Так же можно по визуализации понять, какие гиперпараметры модели более важные.

<img src=table.png width=1450px height=30%>